In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df_demo = pd.read_csv("../data/raw/df_final_demo.txt", sep=",")
df_web_pt1 = pd.read_csv("../data/raw/df_final_web_data_pt_1.txt", sep=",")
df_web_pt2 = pd.read_csv("../data/raw/df_final_web_data_pt_2.txt", sep=",")
df_exp = pd.read_csv("../data/raw/df_final_experiment_clients.txt", sep=",")

In [ ]:
df_web = pd.concat([df_web_pt1, df_web_pt2], axis=0, ignore_index=True)
df_web.head()

In [ ]:
df_exp.head()

### KPI 1: Completion Rate
A visit is considered completed if it contains at least one "confirm" step.
Completion rate is calculated at the visit level.

In [ ]:
df_web = df_web.merge(
    df_exp[["client_id", "Variation"]],
    on="client_id",
    how="inner"
)

In [ ]:
visit_completion = (
    df_web
    .groupby(["visit_id", "Variation"])["process_step"]
    .apply(lambda x: "confirm" in x.values)
    .reset_index(name="completed")
)
visit_completion

In [ ]:
completion_rate = (
    visit_completion
    .groupby("Variation")["completed"]
    .mean()
    .reset_index()
)

completion_rate

The Test variation shows a higher completion rate compared to the Control group, suggesting that the new design may encourage more users to complete the process.

### KPI 2: Time Spent on Each Step

Time spent on each step is calculated as the time difference between consecutive actions within the same visit.
This metric helps evaluate whether the new design enables users to progress through the process more efficiently.

In [ ]:
df_web["date_time"] = pd.to_datetime(df_web["date_time"])

In [ ]:
df_web_sorted = df_web.sort_values(
    ["visit_id", "date_time"]
)

In [ ]:
df_web_sorted["time_diff_sec"] = (
    df_web_sorted
        .groupby("visit_id")["date_time"]
        .diff() # current time - previous time
        .dt.total_seconds() #change time to float eg. 0 days 00:00:20 ->20
)

In [ ]:
df_web_sorted[["visit_id", "process_step", "time_diff_sec"]].head(10)

In [ ]:
avg_time_per_step = (
    df_web_sorted
    .groupby(["Variation", "process_step"])["time_diff_sec"]
    .mean()
    .reset_index()
)

avg_time_per_step

Overall, the time spent per step is fairly similar between the Control and Test variations.
The Test group shows slightly faster completion in step_1, while spending more time on step_2 and step_3.
This suggests that the new design does not substantially reduce time spent across all steps, but may alter how users allocate time within the process.

### KPI 3: Error Rate

An error is defined as a backward movement in the process flow, where a user moves from a later step to an earlier step within the same visit.
This metric helps identify potential confusion or friction in the user journey.

In [ ]:
step_order = {
    "start": 0,
    "step_1": 1,
    "step_2": 2,
    "step_3": 3,
    "confirm": 4
}

df_web_sorted["step_num"] = df_web_sorted["process_step"].map(step_order)

In [ ]:
df_web_sorted["step_diff"] = (
    df_web_sorted
    .groupby("visit_id")["step_num"]
    .diff()
)
# step_diff > 0 → normal
# step_diff = 0 → repeat
# step_diff < 0 → error

In [ ]:
df_web_sorted["error"] = df_web_sorted["step_diff"] < 0

In [ ]:
visit_errors = (
    df_web_sorted
    .groupby(["visit_id", "Variation"])["error"]
    .any()
    .reset_index()
)
visit_errors

In [ ]:
error_rate = (
    visit_errors
    .groupby("Variation")["error"]
    .mean()
    .reset_index()
)

error_rate

The Test variation shows a higher error rate than the Control group, suggesting the new design may introduce more confusion or friction (users more often move backward in the flow).